# Ejercicio Covid-19

In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
#Fechas de hoy(datos España) y ayer (datos inetrnacionales)
from datetime import datetime, timedelta
from datetime import date
date_today = date.today()
date_today_complete = date.today()
date_daybefore = date_today - timedelta(days=1)
date_today = date_today.strftime('%d/%m')
date_daybefore = date_daybefore.strftime('%d/%m')

In [15]:
new_inform = 74
new_before = new_inform - 1
path_spain = 'pdf_files/Actualizacion_74_COVID-19.pdf'
path_spain_before = 'pdf_files/Actualizacion_73_COVID-19.pdf'

date_today_large = date_today_complete.strftime('%Y-%m-%d')
date_today_large

'2020-04-13'

# Preparación inicial

## Descargar PDFs del Ministerio de Sanidad

In [16]:
#Importar PDFs
import os
import requests
from bs4 import BeautifulSoup
import re

#Función para descargar PDFs
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    print(local_filename)
    return local_filename

#
DAILY_FOLDER = "./pdf_files/"
ES_REPORT_URL = "https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/situacionActual.htm"
PDF_BASE_URL = "https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/"

#Función para revisar los nombres de archivos
def get_file_names(url):
    try:
        req_page = requests.get(url, verify=True)
    except Exception as e:
        raise Exception(f"Could not get web page content: {e}")
    
    re_pdf = re.compile(r"(documentos/.+?\.pdf)")

    pdf_paths = list(set(re_pdf.findall(req_page.text)))
    return pdf_paths

#Reviso los nomnbres de archivos
pdf_paths = get_file_names(ES_REPORT_URL)

pdf_paths = [PDF_BASE_URL+i for i in pdf_paths if "Informacion_inicial_alerta" not in i]


for pdf_path in pdf_paths:
        pdf_name = pdf_path.split("/")[-1]
        pdf_path_get = requests.get(pdf_path)
        with open(
            os.path.join(DAILY_FOLDER, pdf_name),
            'wb'
        ) as f:
            f.write(pdf_path_get.content)
            
# Descargo PDFs a raíz
for i in range(new_before,new_inform):
    path = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/documentos/Actualizacion_{}_COVID-19.pdf'.format(i+1)
    download_file(path)

# Muevo los archivos a otra carpeta
lst_files = [file for file in os.listdir() if 'Act' in file]
if os.path.isdir("./pdf_files/"):
    for f in lst_files:
        os.rename("./{}".format(f), "./pdf_files/{}".format(f))
else:
    os.mkdir("./pdf_data/")
    for f in lst_files:
        os.rename("./{}".format(f), "./pdf_files/{}".format(f))

Actualizacion_74_COVID-19.pdf


## Parsear PDF descargado

In [17]:
# Parsear PDF

from tika import parser
import pandas as pd
from io import StringIO
import re
import tabula

raw = parser.from_file(path_spain)

#Cojo partes del PDF
ind_ini = raw['content'].find('en España:')
ind_fin = raw['content'].find('mundo:')


#Coger fecha del PDF
def get_fecha(string):
    ind_ini = string.find('(COVID-19)')
    ind_fin = string.find('.20')
    print(string[ind_ini:ind_fin])
    return string[ind_ini:ind_fin].split('\n')[-1]+'.2020'

fecha_data_spain = get_fecha(raw['content'])
print(fecha_data_spain)

# Coger tablar primera página
import tabula
from tabula import read_pdf
data_spain_ccaa_all = tabula.read_pdf(path_spain, stream=True, pages=1)[0]
data_spain_ccaa_all.columns = ['Comunidades Autónomas','Confirmados', 'IA', 'Hospitalizados', 'UCI', 'Fallecidos', 'Curados', 'Nuevos casos (24h)']
data_spain_ccaa = data_spain_ccaa_all.iloc[3:22]
data_spain_ccaa = data_spain_ccaa[['Comunidades Autónomas', 'Confirmados', 'Curados', 'Fallecidos', 'Nuevos casos (24h)']]

#Quitar símbolos raros qye nos sobran
data_spain_ccaa['Confirmados'] = data_spain_ccaa['Confirmados'].str.replace('.','')
data_spain_ccaa['Fallecidos'] = data_spain_ccaa['Fallecidos'].str.replace('.','')
data_spain_ccaa['Curados'] = data_spain_ccaa['Curados'].str.replace('.','')
data_spain_ccaa['Nuevos casos (24h)'] = data_spain_ccaa['Nuevos casos (24h)'].str.replace('.','')

# Seleccionar índice Comunidades Autónomas
data_spain_ccaa = data_spain_ccaa.set_index('Comunidades Autónomas')
data_spain_ccaa

(COVID-19).  

13.04
13.04.2020


,Confirmados,Curados,Fallecidos,Nuevos casos (24h)
Comunidades Autónomas,,,,
Andalucía,10187,1967,815,181
Aragón,4187,946,464,117
Asturias,1958,434,149,66
Baleares,1550,862,117,16
Canarias,1944,458,96,26
Cantabria,1777,317,117,25
Castilla La Mancha,14054,2532,1626,356
Castilla y León,12628,4154,1263,510
Cataluña,34726,15602,3538,699


# Resumen de datos INICIAL

## Resumen datos globales CSV: covid19_data_drive_resume.csv

In [18]:
# Creación de funciones para sacar datos
def get_data_spain(url):    
    data_spain = pd.read_csv(url)
    data_spain = data_spain.groupby("Country/Region", as_index=False).sum()
    data_spain = data_spain.set_index("Country/Region").transpose()
    data_spain = data_spain['Spain']
    data_spain_actual = int(data_spain.iloc[-1])
    data_spain_before = int(data_spain.iloc[-2])
    data_spain_24h = data_spain_actual - data_spain_before
    return data_spain_24h, data_spain_actual
    
# Función para obtener datos del mundo    
def get_data_world(url):    
    data_world = pd.read_csv(url)
    data_world = data_world.groupby("Country/Region", as_index=False).sum()
    data_world = data_world.set_index("Country/Region").transpose()
    data_world_actual = int(data_world.sum(axis=1).iloc[-1])
    data_world_before = int(data_world.sum(axis=1).iloc[-2])
    data_world_24h = data_world_actual - data_world_before
    return data_world_24h, data_world_actual

# Usar los datos
data_confirmed_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
data_deaths_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
data_recovered_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

# Para España
data_confirmed_spain_24h, data_confirmed_spain_actual = get_data_spain(data_confirmed_path)
data_deaths_spain_24h, data_deaths_spain_actual = get_data_spain(data_deaths_path)
data_recovered_spain_24h, data_recovered_spain_actual = get_data_spain(data_recovered_path)

# Para el mundo
data_confirmed_world_24h, data_confirmed_world_actual = get_data_world(data_confirmed_path)
data_deaths_world_24h, data_deaths_world_actual = get_data_world(data_deaths_path)
data_recovered_world_24h, data_recovered_world_actual = get_data_world(data_recovered_path)

######### Para datos de John Hopkins internacionales

'''data_spain_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_spain_actual,
                                    'Curados': data_deaths_spain_actual, 
                                    'Fallecidos': data_recovered_spain_actual}, 
                                   index=['Casos totales en España'])

data_spain_newcases = pd.DataFrame({'Confirmados': data_confirmed_spain_24h,
                                    'Curados': data_recovered_spain_24h, 
                                    'Fallecidos': data_deaths_spain_24h}, 
                                   index=['Nuevos casos en España (24h)'])
'''

data_world_newcases = pd.DataFrame({'Confirmados': data_confirmed_world_24h,
                                    'Curados': data_recovered_world_24h, 
                                    'Fallecidos': data_deaths_world_24h}, 
                                   index=['Nuevos casos en el mundo (24h)'])

data_world_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_world_actual,
                                    'Curados': data_recovered_world_actual, 
                                    'Fallecidos': data_deaths_world_actual}, 
                                   index=['Casos totales en el mundo'])

########## Para datos de España del Ministerio de Sanidad
# Coger los datos del día anterior
import tabula
from tabula import read_pdf
data_spain_ccaa_daybefore = tabula.read_pdf(path_spain_before, stream=True, pages=1)[0]
data_spain_ccaa_daybefore.columns = ['Comunidades Autónomas','Confirmados', 'IA', 'Hospitalizados', 'UCI', 'Fallecidos', 'Curados', 'Nuevos casos (24h)']

data_spain_ccaa_daybefore =  data_spain_ccaa_daybefore.iloc[22:23].copy()
data_spain_ccaa_daybefore.rename(index={22: "Casos totales en España"}, inplace=True)
data_spain_ccaa_daybefore = data_spain_ccaa_daybefore[['Confirmados', 'Curados', 'Fallecidos']]
data_spain_ccaa_daybefore['Confirmados'] = data_spain_ccaa_daybefore['Confirmados'].str.replace('.','').astype('int')
data_spain_ccaa_daybefore['Fallecidos'] = data_spain_ccaa_daybefore['Fallecidos'].str.replace('.','').astype('int')
data_spain_ccaa_daybefore['Curados'] = data_spain_ccaa_daybefore['Curados'].str.replace('.','').astype('int')

# Coger los datos del día de hoy
data_spain_ccaa_daytoday = data_spain_ccaa_all.iloc[22:23].copy()
data_spain_ccaa_daytoday.rename(index={22: "Casos totales en España"}, inplace=True)
data_spain_ccaa_daytoday = data_spain_ccaa_daytoday[['Confirmados', 'Curados', 'Fallecidos']]
data_spain_ccaa_daytoday['Confirmados'] = data_spain_ccaa_daytoday['Confirmados'].str.replace('.','').astype('int')
data_spain_ccaa_daytoday['Fallecidos'] = data_spain_ccaa_daytoday['Fallecidos'].str.replace('.','').astype('int')
data_spain_ccaa_daytoday['Curados'] = data_spain_ccaa_daytoday['Curados'].str.replace('.','').astype('int')


# Restar datos de dataframes
data_spain_ccaa_daytoday # Muestra los datos del día actual
data_spain_ccaa_daybefore # Muestra los datos del día anterior
data_spain_ccaa_restdays = data_spain_ccaa_daytoday - data_spain_ccaa_daybefore # Muestra los datos en 24h.
data_spain_ccaa_restdays.rename(index={"Casos totales en España": "Nuevos casos en España (24h)"}, inplace=True)
# Aquí fue superimportante ponerlo así, y no así: data_spain_ccaa_restdays.rename(index={0: "Casos totales en España"}, inplace=True), así no funcionaba

########### Junto los datos anteriores en una tabla
data_resume_final = pd.concat([data_world_resume_actual, data_world_newcases, data_spain_ccaa_daytoday, data_spain_ccaa_restdays])

data_resume_final.to_csv('covid19_data_drive_resume.csv', sep = ';')
print(data_resume_final)

Confirmados  Curados  Fallecidos
Casos totales en el mundo           1846679   421722      114091
Nuevos casos en el mundo (24h)        75165    19612        5588
Casos totales en España              169496    64727       17489
Nuevos casos en España (24h)           3477     2336         517


# Datos a nivel mundial

## Preparación general de datos por países

In [19]:
# Datos https://github.com/CSSEGISandData/COVID-19.git
data = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )

#Saco un dataset con solo los países
data_countries = data.groupby("Country/Region", as_index=False).sum()

# Hago transposición de los datos para mover las columnas en filas 
data_trans = data_countries.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_time = data_trans[2:]

# Cogemos solo determinados países
data_time = data_time[['China', 'France', 'Germany', 'Iran','Italy', 'Korea, South', 'Norway','Spain', 'US', 'United Kingdom']]

#Hago listado de Countries
list_countries = data_time.columns
list_countries

#Convierto la columna fecha en datetime
from datetime import date
data_time.index = pd.to_datetime(data_time.index)
data_time.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-04-08,82809.0,113959.0,113296.0,64586.0,139422.0,10384.0,6086.0,148220.0,429052.0,61474.0
2020-04-09,82883.0,118781.0,118181.0,66220.0,143626.0,10423.0,6211.0,153222.0,461437.0,65872.0
2020-04-10,82941.0,125931.0,122171.0,68192.0,147577.0,10450.0,6314.0,158273.0,496535.0,74605.0
2020-04-11,83014.0,130727.0,124908.0,70029.0,152271.0,10480.0,6409.0,163027.0,526396.0,79874.0
2020-04-12,83134.0,133670.0,127854.0,71686.0,156363.0,10512.0,6525.0,166831.0,555313.0,85206.0


## MUNDO: CSV : covid19_data_drive_actual.csv

In [20]:
######### Confirmados

data_confirmed = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )
#Saco un dataset con solo los países
data_confirmed = data_confirmed.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_confirmed = data_confirmed.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_confirmed = data_confirmed[2:]
# Cogemos solo determinados países
data_confirmed = data_confirmed[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_confirmed = data_confirmed[-1:]

######### Muertos

data_deaths = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
#Saco un dataset con solo los países
data_deaths = data_deaths.groupby("Country/Region", as_index=False).sum()
#Se ve que están todos y comprobado que suma
data_deaths[::]
# Hago transposición de los datos para mover las columnas en filas 
data_deaths = data_deaths.set_index('Country/Region').transpose()

# Y me quedo sólo con los datos temporales y número de casos
data_deaths = data_deaths[2:]
data_deaths.tail()
# Cogemos solo determinados países
data_deaths = data_deaths[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_deaths = data_deaths[-1:]

######### Recuperados

data_recovered = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv" )
#Saco un dataset con solo los países
data_recovered = data_recovered.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_recovered = data_recovered.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_recovered = data_recovered[2:]
# Cogemos solo determinados países
data_recovered = data_recovered[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_recovered = data_recovered[-1:]

######### Merge de datos

# Saco la fecha:
from datetime import datetime, timedelta
date_today_intern = date.today()
date_today_intern = date_today_intern.today() - timedelta(days=1) #Quito uno porque esta base de datos siempre muestra datos del día anterior
date_today_intern = date_today_intern.strftime('%d/%m')

#Juntar ahora datos
data_columns = ['Confirmados', 'Curados', 'Fallecidos']

data_together = pd.merge(data_confirmed, data_recovered, how='outer')
data_together = pd.merge(data_together, data_deaths, how='outer')
data_together.index = data_columns
data_together = data_together.transpose()
# data_together = data_together.reindex(columns =data_columns) 
# data_together = data_together.reindex(data_index)

# Exporto CSV
data_together.to_csv('covid19_data_drive_actual.csv', sep = ';')
data_together

,Confirmados,Curados,Fallecidos
Country/Region,,,
China,83134.0,77956.0,3343.0
France,133670.0,27469.0,14412.0
Germany,127854.0,60300.0,3022.0
Italy,156363.0,34211.0,19899.0
Spain,166831.0,62391.0,17209.0
US,555313.0,32988.0,22020.0
United Kingdom,85206.0,626.0,10629.0


## MUNDO: CSV : covid19_data_drive_from100.csv

In [21]:
'''Vamos a hacer un gráfico para ver los casos confirmados (desde caso 100)'''

# Coger solo datos del último mes
data_time_30days = data_time[-200:]
# Si quiero quitar China: data_time_filter = data_time_filter.drop('China', axis=1)

#Exporto los datos en diferentes arhcivos con casa país
for country in list_countries:
    dataX = data_time_30days[country]
    dataX = dataX[dataX > 200]
    dataX.to_csv('data_' + country + ".csv", index=False, header= country)

# Lista dataframes
# dataframes_list = ['dataframe_France', 'dataframe_China', 'dataframe_Germany', 'dataframe_Iran', 'dataframe_Italy', 'dataframe_Korea', 'dataframe_Norway', 'dataframe_Spain', 'dataframe_US', 'dataframe_UnitedKingdom']

# Creamos los CSV
dataframe_France = pd.read_csv('data_France.csv')
dataframe_China = pd.read_csv('data_China.csv')
dataframe_Germany = pd.read_csv('data_Germany.csv')
dataframe_Iran = pd.read_csv('data_Iran.csv')
dataframe_Italy = pd.read_csv('data_Italy.csv')
dataframe_Korea = pd.read_csv('data_Korea, South.csv')
dataframe_Norway = pd.read_csv('data_Norway.csv')
dataframe_Spain = pd.read_csv('data_Spain.csv')
dataframe_US = pd.read_csv('data_US.csv')
dataframe_UnitedKingdom = pd.read_csv('data_United Kingdom.csv')

#Les añadimos NaN al final y reindexamos
new_index = np.arange(1,51)
dataframe_France = dataframe_France.reindex(new_index)
dataframe_China = dataframe_China.reindex(new_index)
dataframe_Germany = dataframe_Germany.reindex(new_index)
dataframe_Iran = dataframe_Iran.reindex(new_index)
dataframe_Italy = dataframe_Italy.reindex(new_index)
dataframe_Korea = dataframe_Korea.reindex(new_index)
dataframe_Norway = dataframe_Norway.reindex(new_index)
dataframe_Spain = dataframe_Spain.reindex(new_index)
dataframe_US = dataframe_US.reindex(new_index)
dataframe_UnitedKingdom = dataframe_UnitedKingdom.reindex(new_index)

#Juntar dataframes y Exportar CSV completo
data_drive_from100cases = dataframe_France.join([dataframe_China, dataframe_Germany, 
                                                 dataframe_Iran, dataframe_Italy, dataframe_Korea, 
                                                 dataframe_Norway, dataframe_Spain, dataframe_US,
                                                 dataframe_UnitedKingdom])

data_drive_from100cases.to_csv('covid19_data_drive_from100.csv', sep = ';')
data_drive_from100cases.tail()

,France,China,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
46,NaN,80823.0,NaN,NaN,147577.0,10331.0,NaN,NaN,NaN,NaN
47,NaN,80860.0,NaN,NaN,152271.0,10384.0,NaN,NaN,NaN,NaN
48,NaN,80887.0,NaN,NaN,156363.0,10423.0,NaN,NaN,NaN,NaN
49,NaN,80921.0,NaN,NaN,NaN,10450.0,NaN,NaN,NaN,NaN
50,NaN,80932.0,NaN,NaN,NaN,10480.0,NaN,NaN,NaN,NaN


## MUNDO: CSV covid19_data_drive_days.csv

In [22]:
# Coger solo datos de últimos 60 días
data_time_Xdays = data_time[-30:]

#Exportar CSV por días progresivos
data_drive_lastX = data_time_Xdays.loc[:,list_countries]
data_drive_lastX.to_csv('covid19_data_drive_days.csv', sep = ';', encoding="utf-8")
data_drive_lastX.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-04-08,82809.0,113959.0,113296.0,64586.0,139422.0,10384.0,6086.0,148220.0,429052.0,61474.0
2020-04-09,82883.0,118781.0,118181.0,66220.0,143626.0,10423.0,6211.0,153222.0,461437.0,65872.0
2020-04-10,82941.0,125931.0,122171.0,68192.0,147577.0,10450.0,6314.0,158273.0,496535.0,74605.0
2020-04-11,83014.0,130727.0,124908.0,70029.0,152271.0,10480.0,6409.0,163027.0,526396.0,79874.0
2020-04-12,83134.0,133670.0,127854.0,71686.0,156363.0,10512.0,6525.0,166831.0,555313.0,85206.0


# Datos a nivel España

## ESPAÑA: Resumen datos España: data_spain_newcases24h.csv

In [23]:
path_c = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
path_d = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
path_r = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
# Evolución últimos 20 días de neuvos casos


def day_consecutive(day):
    date_cons = date.today()
    date_cons = date_cons - timedelta(days=day)
    date_cons = date_cons.strftime('%Y-%m-%d')
    return date_cons

def data_day(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Spain']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_italy(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Italy']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_france(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['France']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_germany(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Germany']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def create_dataframe(data_day_name, data_path, index_title):
    dataframe = pd.DataFrame({day_consecutive(40): data_day_name(data_path, -40),
                              day_consecutive(39): data_day_name(data_path, -39),
                              day_consecutive(38): data_day_name(data_path, -38),
                              day_consecutive(37): data_day_name(data_path, -37),
                              day_consecutive(36): data_day_name(data_path, -36),
                              day_consecutive(35): data_day_name(data_path, -35),
                              day_consecutive(34): data_day_name(data_path, -34),
                              day_consecutive(33): data_day_name(data_path, -33),
                              day_consecutive(32): data_day_name(data_path, -32),
                              day_consecutive(31): data_day_name(data_path, -31),
                              day_consecutive(30): data_day_name(data_path, -30),
                              day_consecutive(29): data_day_name(data_path, -29),
                              day_consecutive(28): data_day_name(data_path, -28),
                              day_consecutive(27): data_day_name(data_path, -27),
                              day_consecutive(26): data_day_name(data_path, -26),
                              day_consecutive(25): data_day_name(data_path, -25),
                              day_consecutive(24): data_day_name(data_path, -24),
                              day_consecutive(23): data_day_name(data_path, -23),
                              day_consecutive(22): data_day_name(data_path, -22),
                              day_consecutive(21): data_day_name(data_path, -21),
                              day_consecutive(20): data_day_name(data_path, -20),
                              day_consecutive(19): data_day_name(data_path, -19), 
                              day_consecutive(18): data_day_name(data_path, -18),
                              day_consecutive(17): data_day_name(data_path, -17),
                              day_consecutive(16): data_day_name(data_path, -16),
                              day_consecutive(15): data_day_name(data_path, -15),
                              day_consecutive(14): data_day_name(data_path, -14),
                              day_consecutive(13): data_day_name(data_path, -13),
                              day_consecutive(12): data_day_name(data_path, -12),
                              day_consecutive(11): data_day_name(data_path, -11),
                              day_consecutive(10): data_day_name(data_path, -10),
                              day_consecutive(9): data_day_name(data_path, -9),
                              day_consecutive(8): data_day_name(data_path, -8),
                              day_consecutive(7): data_day_name(data_path, -7),
                              day_consecutive(6): data_day_name(data_path, -6),
                              day_consecutive(5): data_day_name(data_path, -5),
                              day_consecutive(4): data_day_name(data_path, -4),
                              day_consecutive(3): data_day_name(data_path, -3),
                              day_consecutive(2): data_day_name(data_path, -2),
                              day_consecutive(1): data_day_name(data_path, -1),
                             }, index=index_title)
                                    
    return dataframe

data_spain_confirmed24h = create_dataframe(data_day, path_c, ['Confirmados (24h) en España'])
data_spain_deaths24h = create_dataframe(data_day, path_d, ['Fallecidos (24h) en España'])
data_spain_recovered24h = create_dataframe(data_day, path_r, ['Recuperados (24h) en España'])

#Juntar dataframes
data_spain_newcases24h = pd.concat([data_spain_confirmed24h,data_spain_recovered24h,data_spain_deaths24h], axis=0)
data_spain_newcases24h.to_csv('data_spain_newcases24h.csv', sep = ';')
data_spain_deaths24h.to_csv('data_spain_newcases24h_deaths.csv', sep = ';')

######## Lo mismo en italia
data_italy_confirmed24h = create_dataframe(data_day_italy, path_c, ['Confirmados (24h) en Italia'])
data_italy_deaths24h = create_dataframe(data_day_italy, path_d, ['Fallecidos (24h) en Italia'])
data_italy_recovered24h = create_dataframe(data_day_italy, path_r, ['Recuperados (24h) en Italia'])
data_italy_newcases24h = pd.concat([data_italy_confirmed24h,data_italy_recovered24h,data_italy_deaths24h], axis=0)

######## Lo mismo en Francia
data_france_confirmed24h = create_dataframe(data_day_france, path_c, ['Confirmados (24h) en Francia'])
data_france_deaths24h = create_dataframe(data_day_france, path_d, ['Fallecidos (24h) en Francia'])
data_france_recovered24h = create_dataframe(data_day_france, path_r, ['Recuperados (24h) en Francia'])
data_france_newcases24h = pd.concat([data_france_confirmed24h,data_france_recovered24h,data_france_deaths24h], axis=0)

######## Lo mismo en Alemania
data_germany_confirmed24h = create_dataframe(data_day_germany, path_c, ['Confirmados (24h) en Alemania'])
data_germany_deaths24h = create_dataframe(data_day_germany, path_d, ['Fallecidos (24h) en Alemania'])
data_germany_recovered24h = create_dataframe(data_day_germany, path_r, ['Recuperados (24h) en Alemania'])
data_germany_newcases24h = pd.concat([data_germany_confirmed24h,data_germany_recovered24h,data_germany_deaths24h], axis=0)


###### Juntar dataframes Spain + Italy sólo de fallecidos
data_somecountries_newcases24h_deaths = pd.concat([data_spain_deaths24h, data_italy_deaths24h,
                                                   data_france_deaths24h, data_germany_deaths24h], axis=0)
data_somecountries_newcases24h_deaths.to_csv('data_somecountries_newcases24h.csv', sep = ';')
data_somecountries_newcases24h_deaths


,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,...,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12
Fallecidos (24h) en España,1,1,2,5,7,11,7,19,1,78,...,850,749,694,700,704,747,655,634,525,603
Fallecidos (24h) en Italia,28,41,49,36,133,97,168,196,0,439,...,766,681,525,636,604,542,610,570,619,431
Fallecidos (24h) en Francia,0,2,3,2,8,0,14,15,0,31,...,1122,1054,519,833,1417,544,1341,987,636,561
Fallecidos (24h) en Alemania,0,0,0,0,0,2,0,1,0,4,...,168,169,140,226,206,333,258,160,-31,286


## ESPAÑA: CSV: covid19_spain_ccaa.csv datos principales

In [24]:
#Exportar CSV
data_spain_ccaa.to_csv('covid19_spain_ccaa.csv', sep = ';', encoding="utf-8")
data_spain_ccaa.head(19)

,Confirmados,Curados,Fallecidos,Nuevos casos (24h)
Comunidades Autónomas,,,,
Andalucía,10187,1967,815,181
Aragón,4187,946,464,117
Asturias,1958,434,149,66
Baleares,1550,862,117,16
Canarias,1944,458,96,26
Cantabria,1777,317,117,25
Castilla La Mancha,14054,2532,1626,356
Castilla y León,12628,4154,1263,510
Cataluña,34726,15602,3538,699


## ESPAÑA: CSV: covid19_spain_ccaa_lineal.csv

In [26]:
# Casos confirmados por ccaa temporal

# Cojo la fila de confirmados que me interesa
data_spain_ccaa_last = data_spain_ccaa.copy()
data_spain_ccaa_last.rename(columns = {'Confirmados': date_today_large}, index={'ccaa': 'Comunidades Autónomas'}, inplace = True)
data_spain_ccaa_last = data_spain_ccaa_last[date_today_large]
data_spain_ccaa_last = pd.to_numeric(data_spain_ccaa_last).astype(np.int64)
data_spain_ccaa_last.head()

#HACER MERGE SI NO ESTÁN ESOS DATOS

# Importo los últimos datos recogidos acumulados anteriormente
data_process = pd.read_csv("covid19_spain_ccaa_evolution2.csv", sep = ';', encoding="utf-8")
data_process = data_process.set_index('Comunidades Autónomas')

# Hacer merge de ambos dataframes, con un condicional comprobando si es necesario
if data_process.columns[-1] == data_spain_ccaa_last.name:
    print('La columna ya está: NO hacer merge y no se exporta tabla')
else:
    print('La última columna es diferente y no está: SE HACE MERGE')
    data_process = pd.merge(data_process,data_spain_ccaa_last, left_index=True, right_index=True)
    #Exportar a CSV
    data_process.to_csv('covid19_spain_ccaa_evolution2.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')

data_process

La columna ya está: NO hacer merge y no se exporta tabla


,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,...,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13
Comunidades Autónomas,,,,,,,,,,,,,,,,,,,,,
Andalucía,6,12,13,13,13,21,21,21,54,71,...,7869,8301,8581,8767,8997,9261,9510,9712,10006,10187
Aragón,2,2,2,2,2,6,6,6,13,38,...,3078,3232,3347,3449,3549,3685,3831,3969,4070,4187
Asturias,0,1,1,2,5,5,5,5,7,22,...,1522,1605,1646,1679,1705,1737,1799,1827,1892,1958
Baleares,2,2,2,5,6,6,6,6,11,13,...,1271,1293,1320,1369,1412,1448,1488,1507,1534,1550
Canarias,6,7,7,7,8,11,11,11,22,25,...,1564,1622,1649,1725,1762,1834,1858,1887,1918,1944
Cantabria,0,10,10,10,10,10,10,10,12,12,...,1384,1441,1483,1501,1572,1619,1659,1719,1752,1777
Castilla La Mancha,0,3,7,12,13,15,15,15,26,39,...,9324,10031,10602,11077,11788,12489,13063,13456,13698,14054
Castilla y León,2,3,5,11,11,14,14,14,23,56,...,8332,8749,9116,9581,10058,10518,11102,11543,12118,12628
Cataluña,3,15,15,15,24,24,24,24,75,124,...,24734,26032,26824,28323,29647,31043,31824,32984,34027,34726


## ESPAÑA: Análisis rangos de esas: CSV

In [27]:
data_spain_ccaa_age = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19_rango_edad.csv", 
                                  sep = ',', encoding="utf-8")
data_spain_ccaa_age = data_spain_ccaa_age[data_spain_ccaa_age["fecha"] == date_today_large]

def take_edad(sexo):
    data_spain_ccaa_age_all = data_spain_ccaa_age[data_spain_ccaa_age["sexo"] == sexo]
    #Para sacar datos total
    data_spain_ccaa_age_total = data_spain_ccaa_age_all.tail(1)
    data_spain_ccaa_age_total = data_spain_ccaa_age_total.iloc[:, [3,4,6]]
    data_spain_ccaa_age_total.columns = ['Confirmados','Hospitalizados', 'Fallecidos']
    #Para sacar datos concretos (de ambos, hombres y mujeres)
    data_spain_ccaa_age_concrete = data_spain_ccaa_age_all[:-1]
    data_spain_ccaa_age_concrete = data_spain_ccaa_age_concrete.iloc[:, [1,3,4,6]]
    data_spain_ccaa_age_concrete.columns = ['Rango de edad','Confirmados','Hospitalizados', 'Fallecidos']
    data_spain_ccaa_age_concrete.set_index('Rango de edad', inplace=True)
    return data_spain_ccaa_age_concrete, data_spain_ccaa_age_total


data_spain_ccaa_age_ambos_concrete, data_spain_ccaa_age_ambos_total = take_edad('ambos')
data_spain_ccaa_age_hombres_concrete, data_spain_ccaa_age_hombres_total = take_edad('hombres')
data_spain_ccaa_age_mujeres_concrete, data_spain_ccaa_age_mujeres_total = take_edad('mujeres')


data_spain_ccaa_age_ambos_total.index = ['Total ambos sexos']
data_spain_ccaa_age_hombres_total.index = ['Total hombres']
data_spain_ccaa_age_mujeres_total.index = ['Total mujeres']
data_spain_ccaa_age_ambos_concat = pd.concat([data_spain_ccaa_age_mujeres_total, data_spain_ccaa_age_hombres_total, 
                                             data_spain_ccaa_age_ambos_total], axis=0)

data_spain_ccaa_age_mujeres_concrete.to_csv('covid19_spain_ccaa_ages_mujeres.csv', sep = ';', encoding="utf-8")
data_spain_ccaa_age_hombres_concrete.to_csv('covid19_spain_ccaa_ages_hombres.csv', sep = ';', encoding="utf-8")
data_spain_ccaa_age_ambos_concat.to_csv('covid19_spain_ccaa_ages_ambos.csv', sep = ';', encoding="utf-8")

data_spain_ccaa_age_mujeres_concrete

,Confirmados,Hospitalizados,Fallecidos
Rango de edad,,,
0-9,184,64,1
10-19,341,71,1
20-29,4000,462,5
30-39,6948,1132,15
40-49,9989,2197,39
50-59,12095,3549,89
60-69,9134,4511,259
70-79,7884,5410,770
80-89,8537,4952,1624


# Subir archivos a SERVIDOR desde FTP

In [28]:
import csv
from ftplib import FTP
from io import StringIO

In [29]:
# Ayuda: https://www.pythoncircle.com/post/668/uploading-a-file-to-ftp-server-using-python/

passw = pd.read_csv("pass.txt", sep = ',', encoding="utf-8")
host = "akakicreations.com"
username = passw.columns[0]
password = passw.columns[1]

ftp = FTP(host=host)

login_status = ftp.login(user=username, passwd=password)
print(login_status)

print(ftp.dir())
fp = open("covid19_data_drive_days.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_data_drive_days.csv"), fp, 1024)
fp = open("covid19_data_drive_from100.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_data_drive_from100.csv"), fp, 1024)
fp = open("covid19_spain_ccaa.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_evolution2.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_evolution2.csv"), fp, 1024)
fp = open("covid19_data_drive_actual.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_data_drive_actual.csv"), fp, 1024)
fp = open("covid19_data_drive_resume.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_data_drive_resume.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_ages.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_ages.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_ages_mujeres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_ages_mujeres.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_ages_hombres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_ages_hombres.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_ages_ambos.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_ages_ambos.csv"), fp, 1024)
fp = open("data_spain_newcases24h.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_newcases24h.csv"), fp, 1024)
fp = open("data_spain_newcases24h_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_newcases24h_deaths.csv"), fp, 1024)
fp = open("data_somecountries_newcases24h.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_somecountries_newcases24h.csv"), fp, 1024)
fp.close()


230-OK. Current restricted directory is /
230 318 Kbytes used (0%) - authorized: 51200 Kb
drwxr-xr-x    2 akakicre   akakicre         4096 Apr 11 10:51 .
drwxr-xr-x    2 akakicre   akakicre         4096 Apr 11 10:51 ..
-rw-------    1 akakicre   akakicre           10 Apr 12 13:31 .ftpquota
-rw-r--r--    1 akakicre   akakicre       260069 Apr  3 12:34 Imagen_github.jpg
-rw-r--r--    1 akakicre   akakicre          265 Apr 12 13:31 covid19_data_drive_actual.csv
-rw-r--r--    1 akakicre   akakicre         2772 Apr 12 13:31 covid19_data_drive_days.csv
-rw-r--r--    1 akakicre   akakicre         3427 Apr 12 13:31 covid19_data_drive_from100.csv
-rw-r--r--    1 akakicre   akakicre          216 Apr 12 13:31 covid19_data_drive_resume.csv
-rw-r--r--    1 akakicre   akakicre          588 Apr 12 13:31 covid19_spain_ccaa.csv
-rw-r--r--    1 akakicre   akakicre          235 Apr 12 13:31 covid19_spain_ccaa_ages.csv
-rw-r--r--    1 akakicre   akakicre          137 Apr 12 13:31 covid19_spain_ccaa_ages_a